# NA 568 Mobile Robotics: Methods & Algorithms Winter 2021 -- Homework 2 -- Kalman Filtering (Task 4 in Julia)

University of Michigan\
January 25, 2021

## Task 4 A. (4pts) 

Generate a point cloud representing 10,000 samples from the distribution over the position of
the beacon as measured in 

i) the sensor frame, i.e. (r, θ) space and 

ii) the Cartesian (x, y) coordinate frame.

In other words, generate observations of (range, bearing) and project these points into (x, y)

In [ ]:
using Random, Distributions, PyPlot
Random.seed!(1)

# Mean and covariance of range and bearing measurements
mu_r = 10;
sigma_r = 0.5;
mu_b = 0;
sigma_b = 0.25;

# Samples coordinate in the sensor frame
samples_S = zeros(10000,2);

# Samples coordinate in the Cartesian frame
samples_C = zeros(10000,2); 


# Plot samples
figure(1)
scatter(vec(samples_S[:,1]),vec(samples_S[:,2]),s=1);
title("Sensor Frame Point Cloud");
xlabel("Range(m)");
ylabel("Bearing(rad)");

figure(2)
scatter(vec(samples_C[:,1]),vec(samples_C[:,2]),s=1);
title("Cartesian Frame Point Cloud");
xlabel("Range(m)");
ylabel("Bearing(rad)");

## Task 4 B. (4 pts) 
What is the (linearized) covariance of the beacon position in (x, y) coordinates? In other words,
write the covariance of an observation in (x, y) coordinates in terms of the covariance of the observation in (range, bearing) coordinates. The transformation is non-linear, so you will need to compute
a first-order approximation (Taylor expansion) of the transformation function. Make the appropriate
Jacobians easy to read in your source code, using comments if necessary.

In [ ]:
# covariance in the sensor frame
Sigma_S = [sigma_r^2 0.0;0.0 sigma_b^2];

# Compute Jacobian
J = zeros(2,2);

# Compute covariance in the Cartesian frame
Sigma_C = zeros(2,2);

println("Jacobian: ",J);
println("Covariance in the Cartesian frame: ",Sigma_C);

## Task 4 C. (4 pts)
Draw in red the 1-sigma, 2-sigma, and 3-sigma contours of the analytical (linearized) covariance
ellipses, super-imposed over the point clouds generated in parts A.i and A.ii. Now overlay in black the
actual covariance ellipses computed using sample-based expressions for the first and second moments.
Do they agree? Why or why not?

Drawing function draw_ellipse() and codes are provided. 

In [ ]:
using LinearAlgebra

function calculateEllipseXY(mu, Sigma, k2, N)
# mu     is the [2x1] [x;y] mean vector
# Sigma  is the [2x2] covariance matrix
# k2     is the Chi-squared 2 DOF variable
# N      is the number of points to use
    theta = collect(LinRange(0,2*pi,N));
    circle = [cos.(theta) sin.(theta)]';
    
    # Cholesky decomposition
    F = cholesky(Sigma);
    
    # map unit circular covariance to k2 Sigma ellipse
    Y = sqrt(k2)*F.L*circle;
    
    # shift origin to the mean
    XY = [mu[1]*ones(1,size(Y,2));mu[2]*ones(1,size(Y,2))]+Y;
    
    return XY;
end

function draw_ellipse(mu, Sigma, k2, is_red, label)
# mu     is the [2x1] [x;y] mean vector
# Sigma  is the [2x2] covariance matrix
# k2     is the Chi-squared 2 DOF variable
# N      is the number of points to use
# is_red if true plot in red, otherwise in black
# label  if true label the curve

# Draws an ellipse centered at mu with covariance Sigma and
# confidence region k2, i.e.,
# K2 = 1; % 1-sigma
# K2 = 4; % 2-sigma
# K2 = 9; % 3-sigma
# K2 = chi2inv(.50, 2); 50% probability contour
    
    N = 20;
    points = calculateEllipseXY(mu, Sigma, k2, N);
    if is_red
        if label
            plot(vec(points[1,:]),vec(points[2,:]),color=:red,label="analytical contours");
        else
            plot(vec(points[1,:]),vec(points[2,:]),color=:red);
        end
    else
        if label
            plot(vec(points[1,:]),vec(points[2,:]),color=:black,label="sample contours");
        else
            plot(vec(points[1,:]),vec(points[2,:]),color=:black);
        end
    end
end

# Compute the mean and covariance of samples in the sensor frame
mu_samples_S = zeros(2,1);
Sigma_samples_S = zeros(2,2);


# Compute the mean and covariance of samples in the Cartesian frame
mu_samples_C = zeros(2,1);
Sigma_samples_C = zeros(2,2);


# Draw analytical and sample covariance contours on data 
figure(1)
scatter(vec(samples_S[:,1]),vec(samples_S[:,2]),s=1,label="data");
mu_S = [mu_r;mu_b];
for i = 1:3
    if i < 3
        draw_ellipse(mu_S,Sigma_S,i^2,true,false);
        draw_ellipse(mu_samples_S,Sigma_samples_S,i^2,false,false);
    else
        draw_ellipse(mu_S,Sigma_S,i^2,true,true);
        draw_ellipse(mu_samples_S,Sigma_samples_S,i^2,false,true);
    end
end
title("Sensor Frame Point Cloud");
xlabel("Range(m)");
ylabel("Bearing(rad)");
legend()

figure(2)
scatter(vec(samples_C[:,1]),vec(samples_C[:,2]),s=1,label="data");
mu_C = [mu_r*cos(mu_b);mu_r*sin(mu_b)];
for i = 1:3
    if i < 3 
        draw_ellipse(mu_C,Sigma_C,i^2,true,false);
        draw_ellipse(mu_samples_C,Sigma_samples_C,i^2,false,false);
    else
        draw_ellipse(mu_C,Sigma_C,i^2,true,true);
        draw_ellipse(mu_samples_C,Sigma_samples_C,i^2,false,true);
    end
end
title("Cartesian Frame Point Cloud");
xlabel("Range(m)");
ylabel("Bearing(rad)");
legend()

## Task 4 D. (4 pts)
From a purely theoretical perspective, assuming that the underlying process is truly Gaussian,
we expect 39.35% of all samples to lie within the 1-sigma contour, 86.47% of samples to lie within the
2-sigma contour, and 98.89% to lie within the 3-sigma contour. (These frequencies were computed
using the cumulative chi-square distribution for two degrees-of-freedom.)
Modify your software to count the samples falling within each (analytical) ellipse for parts A.i and A.ii.
The error of a particular sample
x, measured in “units” of sigma, is known as the Mahalanobis distance.

In [ ]:
using Printf

Count_S = zeros(3,1); # count results of samples in the sensor frame
Count_C = zeros(3,1); # count results of samples in the Cartesian frame

# Compute the Mahalabobis distance of samples


# Display results
Gaussian = [0.39;0.86;0.99];
println("         Sensor frame  Cartesian frame  Gaussian");
for i=1:3
    @printf("%d-sigma%10.2f%16.2f%13.2f\n",i,Count_S[i]/size(samples_S,1),Count_C[i]/size(samples_C,1),Gaussian[i]);
end

## Task 4 F. (5 pts)
Suppose now that the (range, bearing) measurements are not independent but instead jointly correlated under the following three scenarios: a) ρ(rθ) = 0.1, b) ρ(rθ) = 0.5, and c) ρ(rθ) = 0.9. Repeat parts A and C.

In [ ]:
rou = 0.9; # correlation coefficient, eg. 0.1, 0.5, 0.9

# Compute the covariance in the sensor frame
Sigma_S_corr = zeros(2,2); 


# Samples coordinate in the sensor frame
samples_S_corr = zeros(10000,2);


# Compute the covariance in the Cartesian frame
Sigma_C_corr = zeros(2,2); 


# Samples coordinate in the Cartesian frame
samples_C_corr = zeros(10000,2);


# Compute the mean and covariance of samples in the sensor frame
mu_samples_S_corr = zeros(2,1);
Sigma_samples_S_corr = zeros(2,2);


# Compute the mean and covariance of samples in the Cartesian frame
mu_samples_C_corr = zeros(2,1);
Sigma_samples_C_corr = zeros(2,2);


# Draw analytical and sample covariance contours on data 
figure(1)
scatter(vec(samples_S_corr[:,1]),vec(samples_S_corr[:,2]),s=1,label="data");
for i = 1:3
    if i < 3
        draw_ellipse(mu_S,Sigma_S_corr,i^2,true,false);
        draw_ellipse(mu_samples_S_corr,Sigma_samples_S_corr,i^2,false,false);
    else
        draw_ellipse(mu_S,Sigma_S_corr,i^2,true,true);
        draw_ellipse(mu_samples_S_corr,Sigma_samples_S_corr,i^2,false,true);
    end
end
title("Sensor Frame Point Cloud");
xlabel("Range(m)");
ylabel("Bearing(rad)");
legend()

figure(2)
scatter(vec(samples_C_corr[:,1]),vec(samples_C_corr[:,2]),s=1,label="data");
for i = 1:3
    if i < 3 
        draw_ellipse(mu_C,Sigma_C_corr,i^2,true,false);
        draw_ellipse(mu_samples_C_corr,Sigma_samples_C_corr,i^2,false,false);
    else
        draw_ellipse(mu_C,Sigma_C_corr,i^2,true,true);
        draw_ellipse(mu_samples_C_corr,Sigma_samples_C_corr,i^2,false,true);
    end
end
title("Cartesian Frame Point Cloud");
xlabel("Range(m)");
ylabel("Bearing(rad)");
legend()